In [103]:
!pip install sqlalchemy

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 1.4 MB/s eta 0:00:000:00:010:00:01
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)


In [109]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:01:01


## Extract: Bucket S3

### Transformation: Apply business rule

### Loading: Postgres Database

In [107]:
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import os
import logging
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine

### Listing Buckets

In [ ]:
# Instanciar s3 e verificar as funções disponíveis para essa instância
s3 = boto3.client('s3')

dir(s3)

In [10]:
# Listando os buckets da minha conta AWS
response = s3.list_buckets()

print('Existing buckets: ')
for bucket in response["Buckets"]:
    print(f"    {bucket['Name']}")

Existing buckets: 
    bucket-testing-dnc-btfg


In [3]:
# Função para Upload de arquivo no Bucket na AWS
def upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        # Definindo o nome do arquivo conforme está na base
        # Sem criar um novo nome para aws
        object_name = os.path.basename(file_name)

    # Instanciar o boto3
    s3_client = boto3.client('s3')

    try:
        s3_client.upload_file(
            file_name, bucket, object_name
        )
    except ClientError as e:
        logging.error(e)
        return False
    
    return True

In [4]:
# Lista de arquivos e seus respectivos caminhos no bucket
files_to_upload = [
    {
        "file_name": "../input-data/Engenharia-Dados/cars/car-companies-by-country-2024.csv",
        "object_name": "raw/cars/car-companies-by-country-2024.csv"
    },
    {
        "file_name": "../input-data/Engenharia-Dados/cars/Sport car price.csv",
        "object_name": "raw/cars/Sport car price.csv"
    }
]

bucket_name = "bucket-testing-dnc-btfg"

# Iterar sobre a lista de arquivos e fazer o upload de cada um
for file_info in files_to_upload:
    upload_file(file_info["file_name"], bucket_name, file_info["object_name"])

In [5]:
type(files_to_upload)

list

### Listing folders from a bucket

In [13]:
# Listando diretórios dentro de um bucket na AWS
bucket_name = "bucket-testing-dnc-btfg"
prefix = ''
delimiter = '/'

response = s3.list_objects_v2(
    Bucket=bucket_name,
    Prefix=prefix,
    Delimiter = delimiter
)

folders = [prefix['Prefix'] for prefix in response.get('CommonPrefixes', [])]

folders

['raw/']

### Listar objetos dentro do bucket

In [16]:
# Listar os objetos dentro de um bucket

bucket_name = 'bucket-testing-dnc-btfg'

response = s3.list_objects_v2(
    Bucket = bucket_name
)

objects = response.get('Contents', [])

for object in objects:
    print(object["Key"])

raw/cars/Sport car price.csv
raw/cars/car-companies-by-country-2024.csv


### Carregar dataset com dados da AWS

In [38]:
# Bucket name
bucket_name = 'bucket-testing-dnc-btfg'
file_key = 'raw/cars/Sport car price.csv'

response = s3.get_object(Bucket=bucket_name, Key=file_key)
csv_content = response['Body'].read().decode('utf-8')

df_portfolio = pd.read_csv(StringIO(csv_content))

df_portfolio.sample(10)

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
873,Dodge,Challenger SRT Hellcat Redeye,2022,6.2,797,707,3.5,"80,190"
613,BMW,M4,2021,3,473,406,4,"72,800"
739,Mercedes-Benz,SLS AMG,2015,6.2,622,468,3.5,"222,500"
737,McLaren,GT,2022,4,620,465,3.1,"211,300"
63,Audi,RS5,2021,2.9,444,443,3.6,"75,400"
611,Audi,RS 7,2022,4,591,590,3.5,"117,000"
29,Ferrari,SF90 Stradale,2021,4,986,590,2.5,"625,000"
225,Audi,R8 Spyder,2022,5.2,562,406,3.5,"183,300"
474,Porsche,Panamera Turbo S,2022,4,630,604,2.9,"180,800"
800,McLaren,570S,2021,3.8,562,443,3.1,"193,000"


In [39]:
df_portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Car Make                 1007 non-null   object
 1   Car Model                1007 non-null   object
 2   Year                     1007 non-null   int64 
 3   Engine Size (L)          997 non-null    object
 4   Horsepower               1007 non-null   object
 5   Torque (lb-ft)           1004 non-null   object
 6   0-60 MPH Time (seconds)  1007 non-null   object
 7   Price (in USD)           1007 non-null   object
dtypes: int64(1), object(7)
memory usage: 63.1+ KB


In [27]:
# Garantir que a coluna seja convertida para string primeiro, caso necessário
df_portfolio['Price (in USD)'] = df_portfolio['Price (in USD)'].str.replace(',', '').astype(float)


df_portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Car Make                 1007 non-null   object 
 1   Car Model                1007 non-null   object 
 2   Year                     1007 non-null   int64  
 3   Engine Size (L)          997 non-null    object 
 4   Horsepower               1007 non-null   object 
 5   Torque (lb-ft)           1004 non-null   object 
 6   0-60 MPH Time (seconds)  1007 non-null   object 
 7   Price (in USD)           1007 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 63.1+ KB


In [40]:
# Verificando quantidade de carros por ano
df_portfolio['Year'].value_counts()

Year
2021    576
2022    382
2020     25
2015     12
2019      3
2017      3
2023      3
2014      2
1965      1
Name: count, dtype: int64

In [41]:
# Verificando os carros mais caros
df_portfolio['Price (in USD)'].value_counts().sort_values(ascending=False)

Price (in USD)
500,000      34
3,000,000    24
625,000      22
58,900       17
114,000      16
             ..
49,000        1
45,690        1
57,000        1
132,000       1
35,000        1
Name: count, Length: 367, dtype: int64

In [42]:
# contagem de montadoras:
df_portfolio['Car Make'].value_counts()

Car Make
Porsche          88
McLaren          75
Audi             71
Lamborghini      66
BMW              63
Chevrolet        60
Ferrari          55
Mercedes-Benz    54
Aston Martin     50
Ford             48
Dodge            41
Nissan           37
Lotus            35
Jaguar           30
Lexus            26
Maserati         25
Bentley          25
Bugatti          23
Tesla            19
Alfa Romeo       16
Acura            16
Koenigsegg       15
Rimac            14
Pagani           12
Mercedes-AMG     11
Rolls-Royce      10
Toyota            5
Subaru            3
W Motors          3
TVR               2
Pininfarina       2
Ariel             1
Mazda             1
Polestar          1
Shelby            1
Kia               1
Alpine            1
Ultima            1
Name: count, dtype: int64

In [68]:
# Carregar segundo dataset
# Bucket name
bucket_name = 'bucket-testing-dnc-btfg'
file_key = 'raw/cars/car-companies-by-country-2024.csv'

response = s3.get_object(Bucket=bucket_name, Key=file_key)
csv_content = response['Body'].read().decode('utf-8')

df_country = pd.read_csv(StringIO(csv_content))

df_country.sample(10)

,country,CarCompaniesNumOfCarCompanies,CarCompaniesMostPopularBrands
36,Portugal,2,"Irmaos Mota, UMM"
30,Romania,3,Dacia
19,Spain,20,"Cupra, SEAT"
27,Australia,9,Holden
50,Lithuania,2,"Altas, Forveda"
31,Netherlands,9,Lightyear
55,Liechtenstein,1,NanoFlowcell
2,United States,231,"Buick, Cadillac, Chevrolet, Chrysler, Dodge, F..."
8,Japan,23,"Mazda, Mitsubishi, Nissan, Subaru, Suzuki, Toy..."
40,Austria,9,KTM


In [69]:
# Pegando um registro para analisar
df_country.iloc[7]

country                                            Mexico
CarCompaniesNumOfCarCompanies                           3
CarCompaniesMostPopularBrands    Inferno, Mastretta, VUHL
Name: 7, dtype: object

In [70]:
# Tratando texto da coluna Car...Brands
# Transformando a coluna em listas
df_country['CarCompaniesMostPopularBrands'] = df_country['CarCompaniesMostPopularBrands'].str.split(',')
df_country

,country,CarCompaniesNumOfCarCompanies,CarCompaniesMostPopularBrands
0,India,7,"[Royal Enfield, Tata Motors, TVS, Force Mot..."
1,China,124,"[Chery Automobile, Changan Automobile, CFMot..."
2,United States,231,"[Buick, Cadillac, Chevrolet, Chrysler, Dod..."
3,Pakistan,1,[Adam]
4,Nigeria,1,[Innoson]
5,Brazil,10,"[MP Lafer, Troller]"
6,Russia,24,"[AvtoVAZ, Lada, UAZ, Moskvich]"
7,Mexico,3,"[Inferno, Mastretta, VUHL]"
8,Japan,23,"[Mazda, Mitsubishi, Nissan, Subaru, Suzuki..."
9,Vietnam,1,[VinFast]


In [71]:
df_country.iloc[1]

country                                                                      China
CarCompaniesNumOfCarCompanies                                                  124
CarCompaniesMostPopularBrands    [Chery Automobile,  Changan Automobile,  CFMot...
Name: 1, dtype: object

In [72]:
# Deixando montadoras por linha e renomeando coluna
df_country = df_country.explode('CarCompaniesMostPopularBrands').reset_index(drop=True)
df_country.rename(columns={'CarCompaniesMostPopularBrands': 'Car Make'}, inplace=True)
df_country

,country,CarCompaniesNumOfCarCompanies,Car Make
0,India,7,Royal Enfield
1,India,7,Tata Motors
2,India,7,TVS
3,India,7,Force Motors
4,China,124,Chery Automobile
...,...,...,...
146,Estonia,2,Nobe
147,Estonia,2,TARK
148,Andorra,1,TransAm Depot
149,Liechtenstein,1,NanoFlowcell


In [91]:
# Filtrando somente china em country
df_country[df_country['country']=='Japan']

,country,CarCompaniesNumOfCarCompanies,Car Make,Key
37,Japan,23,Mazda,porsche
38,Japan,23,Mitsubishi,lamborghini
39,Japan,23,Nissan,ferrari
40,Japan,23,Subaru,audi
41,Japan,23,Suzuki,mclaren
42,Japan,23,Toyota,bmw
43,Japan,23,Honda,mercedes-benz
44,Japan,23,Isuzu UTE,porsche
45,Japan,23,Lexus,lamborghini


In [74]:
df_portfolio.sample(2)

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD),Key
33,Mercedes-Benz,SLS AMG,2015,6.2,622,468,3.2,"222,000",mercedes-benz
255,Chevrolet,Camaro SS,2021,6.2,455,455,4.1,"40,590",chevrolet


In [85]:
df_portfolio['Key'] = df_portfolio['Car Make'].str.lower().str.replace(' ', '').str.replace('[^a-zA-Z0-9]', '')
df_country['Key'] = df_portfolio['Car Make'].str.lower().str.replace(' ', '').str.replace('[^a-zA-Z0-9]', '')

In [86]:
df_country.sample(2)

,country,CarCompaniesNumOfCarCompanies,Car Make,Key
73,South Africa,3,CAV,nissan
115,Czech Republic,8,Jawa Motorcycles,mclaren


In [87]:
df_portfolio.sample(2)

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD),Key
918,Lamborghini,Aventador,2021,6.5,729,509,2.9,"393,695",lamborghini
316,McLaren,765LT,2021,4,755,590,2.8,"358,000",mclaren


In [88]:
# Criando novo df fazendo merge dos dois dfs pela coluna key criada
df_merged = pd.merge(df_portfolio, df_country, on='Key', how='left')
df_merged

,Car Make_x,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD),Key,country,CarCompaniesNumOfCarCompanies,Car Make_y
0,Porsche,911,2022,3,379,331,4,"101,200",porsche,India,7.0,Royal Enfield
1,Porsche,911,2022,3,379,331,4,"101,200",porsche,United States,231.0,Cadillac
2,Porsche,911,2022,3,379,331,4,"101,200",porsche,Nigeria,1.0,Innoson
3,Porsche,911,2022,3,379,331,4,"101,200",porsche,Japan,23.0,Mazda
4,Porsche,911,2022,3,379,331,4,"101,200",porsche,Japan,23.0,Isuzu UTE
...,...,...,...,...,...,...,...,...,...,...,...,...
8063,McLaren,Senna,2021,4,789,590,2.7,"1,000,000",mclaren,Lithuania,2.0,Forveda
8064,Pagani,Huayra,2021,6,764,738,3,"2,600,000",pagani,United States,231.0,Jeep
8065,Pagani,Huayra,2021,6,764,738,3,"2,600,000",pagani,Canada,17.0,Ski Doo
8066,Rimac,Nevera,2021,Electric Motor,1888,1696,1.85,"2,400,000",rimac,Pakistan,1.0,Adam


In [89]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Car Make_x                     8068 non-null   object 
 1   Car Model                      8068 non-null   object 
 2   Year                           8068 non-null   int64  
 3   Engine Size (L)                8024 non-null   object 
 4   Horsepower                     8068 non-null   object 
 5   Torque (lb-ft)                 8063 non-null   object 
 6   0-60 MPH Time (seconds)        8068 non-null   object 
 7   Price (in USD)                 8068 non-null   object 
 8   Key                            8068 non-null   object 
 9   country                        8057 non-null   object 
 10  CarCompaniesNumOfCarCompanies  8057 non-null   float64
 11  Car Make_y                     8057 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage

In [90]:
df_merged['country'].value_counts()

country
Japan                   626
China                   557
United Kingdom          471
Italy                   467
Germany                 379
India                   280
Russia                  263
United States           254
United Arab Emirates    222
France                  196
South Africa            179
Turkey                  177
Norway                  159
Canada                  154
Portugal                154
Iran                    146
Lithuania               146
Mexico                  145
Denmark                 143
Israel                  141
South Korea             139
Switzerland             129
Taiwan                  126
Estonia                 125
Serbia                  123
Brazil                  121
Slovenia                117
Sweden                  110
Czech Republic          110
Latvia                  108
Finland                 104
Malaysia                 96
Nigeria                  88
Argentina                86
Poland                   76
Sri Lanka   

In [93]:
df_merged.columns

Index(['Car Make_x', 'Car Model', 'Year', 'Engine Size (L)', 'Horsepower',
       'Torque (lb-ft)', '0-60 MPH Time (seconds)', 'Price (in USD)', 'Key',
       'country', 'CarCompaniesNumOfCarCompanies', 'Car Make_y'],
      dtype='object')

In [94]:
# Elininando colunas desnecessária no contexto
df_merged = df_merged[[
    'Car Make_x',
    'Car Model',
    'Year',
    'Price (in USD)',
    'country']]

In [95]:
df_merged

,Car Make_x,Car Model,Year,Price (in USD),country
0,Porsche,911,2022,"101,200",India
1,Porsche,911,2022,"101,200",United States
2,Porsche,911,2022,"101,200",Nigeria
3,Porsche,911,2022,"101,200",Japan
4,Porsche,911,2022,"101,200",Japan
...,...,...,...,...,...
8063,McLaren,Senna,2021,"1,000,000",Lithuania
8064,Pagani,Huayra,2021,"2,600,000",United States
8065,Pagani,Huayra,2021,"2,600,000",Canada
8066,Rimac,Nevera,2021,"2,400,000",Pakistan


In [96]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Car Make_x      8068 non-null   object
 1   Car Model       8068 non-null   object
 2   Year            8068 non-null   int64 
 3   Price (in USD)  8068 non-null   object
 4   country         8057 non-null   object
dtypes: int64(1), object(4)
memory usage: 315.3+ KB


In [97]:
df_merged['Price (in USD)'] = df_merged['Price (in USD)'].str.replace(',', '').astype(float)

/tmp/ipykernel_9007/3027771327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Price (in USD)'] = df_merged['Price (in USD)'].str.replace(',', '').astype(float)


In [98]:
df_merged

,Car Make_x,Car Model,Year,Price (in USD),country
0,Porsche,911,2022,101200.0,India
1,Porsche,911,2022,101200.0,United States
2,Porsche,911,2022,101200.0,Nigeria
3,Porsche,911,2022,101200.0,Japan
4,Porsche,911,2022,101200.0,Japan
...,...,...,...,...,...
8063,McLaren,Senna,2021,1000000.0,Lithuania
8064,Pagani,Huayra,2021,2600000.0,United States
8065,Pagani,Huayra,2021,2600000.0,Canada
8066,Rimac,Nevera,2021,2400000.0,Pakistan


In [99]:
# Filtrando carros = ou > que 2020
# Filtrando preços = ou > que 100000
# Eliminando linhas que não tem país

df_merged_filtered = df_merged[df_merged['Year'] >= 2020]
df_merged_filtered = df_merged_filtered[df_merged_filtered['Price (in USD)'] >= 100000]
df_merged_filtered = df_merged_filtered.dropna(subset=['country'], axis=0)

In [100]:
df_merged_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5081 entries, 0 to 8067
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Car Make_x      5081 non-null   object 
 1   Car Model       5081 non-null   object 
 2   Year            5081 non-null   int64  
 3   Price (in USD)  5081 non-null   float64
 4   country         5081 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 238.2+ KB


In [101]:
df_merged_filtered

,Car Make_x,Car Model,Year,Price (in USD),country
0,Porsche,911,2022,101200.0,India
1,Porsche,911,2022,101200.0,United States
2,Porsche,911,2022,101200.0,Nigeria
3,Porsche,911,2022,101200.0,Japan
4,Porsche,911,2022,101200.0,Japan
...,...,...,...,...,...
8063,McLaren,Senna,2021,1000000.0,Lithuania
8064,Pagani,Huayra,2021,2600000.0,United States
8065,Pagani,Huayra,2021,2600000.0,Canada
8066,Rimac,Nevera,2021,2400000.0,Pakistan


In [110]:
# Enviando para um banco postgres
username = 'postgres'
password = 'dnc123'
host = 'localhost'
port = 5432
database_name = 'postgres'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database_name}')


In [111]:
df_merged_filtered.to_sql('cars_by_countries_to_sales_campaign', con=engine, if_exists='replace', index=False)

81